<a href="https://colab.research.google.com/github/OverCat2000/BigData/blob/main/BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [101]:
packages = c("sparklyr", "tidyverse", "httr", "jsonlite", "arrow")

package.check <- lapply(
  packages,
  FUN = function(x) {
    if (!require(x, character.only = TRUE)) {
      install.packages(x, dependencies = TRUE)
      library(x, character.only = TRUE)
    }
  }
)

Loading required package: arrow


Attaching package: ‘arrow’


The following object is masked from ‘package:lubridate’:

    duration


The following object is masked from ‘package:utils’:

    timestamp




In [21]:
spark_install(version = 3.5)

In [22]:
sc = spark_connect(master="local")

In [94]:
api_endpoint = "https://data.cityofnewyork.us/resource/2upf-qytp.json?"
limit <- 1000  # Number of rows per request
offset <- 0     # Initial offset

repeat {
  # Construct URL with current offset
  url <- modify_url(api_endpoint, query = list("$limit" = limit, "$offset" = offset))

  # Make the request
  response <- GET(url)

  if (http_error(response)) {
    stop("HTTP request failed: ", httr::http_status(response)$reason)
  }

  data <- as_data_frame(fromJSON(content(response, "text")))

  if (offset != 0) {
    all_data <- bind_rows(all_data, data)
  } else {
    all_data = data
  }

  offset <- offset + limit

  if (dim(all_data)[1] >= 50000) {
    break  # Exit loop if less than 'limit' rows were returned
  }
}

In [105]:
nyc_tbl = copy_to(sc, all_data, "nyc_taxi", overwrite=T)

In [106]:
nyc_tbl

Warning message in arrow_collect(object, ...):
“NAs introduced by coercion to integer range”


# Source:   table<`nyc_taxi`> [?? x 18]
# Database: spark_connection
   vendorid tpep_pickup_datetime    tpep_dropoff_datetime   passenger_count
   <chr>    <chr>                   <chr>                   <chr>          
 1 1        2019-12-04T09:53:38.000 2019-12-04T10:24:54.000 1              
 2 1        2019-12-04T09:38:13.000 2019-12-04T09:59:02.000 1              
 3 2        2019-12-04T09:20:42.000 2019-12-04T09:51:10.000 3              
 4 2        2019-12-04T09:56:30.000 2019-12-04T10:01:00.000 3              
 5 2        2019-12-04T09:01:49.000 2019-12-04T09:38:04.000 1              
 6 2        2019-12-04T09:39:16.000 2019-12-04T09:51:12.000 1              
 7 1        2019-12-04T09:06:51.000 2019-12-04T09:17:47.000 1              
 8 1        2019-12-04T09:31:04.000 2019-12-04T09:42:49.000 1              
 9 1        2019-12-04T09:46:01.000 2019-12-04T09:56:22.000 1              
10 2        2019-12-04T09:12:41.000 2019-12-04T10:00:18.000 1              
# ℹ more rows
# ℹ 1